In [1]:
import os
import duckdb
from dotenv import load_dotenv
import keplergl

In [2]:
load_dotenv()

True

In [3]:
con = duckdb.connect(":memory:")

In [4]:
con.execute("INSTALL spatial; LOAD spatial");
con.execute("INSTALL azure; LOAD azure;");

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
con.execute("SET azure_storage_connection_string = ?;", [os.getenv("BLOB_STORAGE_CONNECTION_STRING")])
con.execute("SET azure_transport_option_type = curl")

In [6]:
con.sql("""
    SELECT * FROM read_parquet('az://raw/release/2025-11-06.3/dataset=osm/theme=buildings/region=18/**.parquet') LIMIT 10
""")

┌──────────────┬────────────────┬───────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [7]:

kart = keplergl.KeplerGl(height=700)

kart.add_data(
    con.sql("""
        SELECT * EXCLUDE(geometry), 
               ST_AsText(geometry) as geometry 
        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=fkb/theme=buildings/region=46/**.parquet')
    """).df(), 
    name='FKB Buildings'
)

kart.add_data(
    con.sql("""
        SELECT * EXCLUDE(geometry), 
               ST_AsText(geometry) as geometry 
        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=osm/theme=buildings/region=46/**.parquet')
    """).df(), 
    name='OSM Buildings'
)

kart


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/usr/local/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

/usr/local/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'FKB Buildings': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1…

In [8]:
# Filter OSM buildings: remove if very similar to FKB, keep if significantly different
# Uses grid-based prefiltering for efficiency (data already in EPSG:4326)
con.execute("""
    CREATE OR REPLACE TABLE filtered_buildings AS
    WITH fkb AS (
        SELECT 
            geometry,
            CAST(FLOOR(ST_X(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_x,
            CAST(FLOOR(ST_Y(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_y,
            ST_Area(geometry) as area
        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=fkb/theme=buildings/region=46/**.parquet')
    ),
    osm AS (
        SELECT 
            geometry,
            CAST(FLOOR(ST_X(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_x,
            CAST(FLOOR(ST_Y(ST_Centroid(geometry)) * 100) AS INTEGER) as grid_y,
            ST_Area(geometry) as area
        FROM read_parquet('az://raw/release/2025-11-06.4/dataset=osm/theme=buildings/region=46/**.parquet')
    ),
    -- Find OSM buildings that intersect with FKB buildings (prefiltered by grid cell)
    osm_with_overlap AS (
        SELECT 
            o.geometry as osm_geom,
            o.area as osm_area,
            f.area as fkb_area,
            ST_Area(ST_Intersection(o.geometry, f.geometry)) / LEAST(o.area, f.area) as overlap_ratio
        FROM osm o
        JOIN fkb f ON o.grid_x = f.grid_x AND o.grid_y = f.grid_y  -- Prefilter using grid
        WHERE ST_Intersects(o.geometry, f.geometry)
    )
    -- Keep all FKB buildings (red)
    SELECT 
        geometry,
        'fkb' as source,
        1 as color
    FROM fkb
    
    UNION ALL
    
    -- Keep OSM buildings with no overlap (green)
    SELECT 
        o.geometry,
        'osm_no_overlap' as source,
        2 as color
    FROM osm o
    WHERE NOT EXISTS (
        SELECT 1 FROM fkb f 
        WHERE f.grid_x = o.grid_x AND f.grid_y = o.grid_y 
        AND ST_Intersects(o.geometry, f.geometry)
    )
    
    UNION ALL
    
    -- Keep OSM buildings with significant difference (overlap < 70%, yellow)
    SELECT 
        osm_geom as geometry,
        'osm_different' as source,
        3 as color
    FROM osm_with_overlap
    WHERE overlap_ratio < 0.70
""")

# Show summary
con.sql("""
    SELECT 
        source,
        COUNT(*) as count,
        CASE source
            WHEN 'fkb' THEN 'FKB buildings (red)'
            WHEN 'osm_no_overlap' THEN 'OSM buildings - no FKB overlap (green)'
            WHEN 'osm_different' THEN 'OSM buildings - significantly different from FKB (yellow)'
        END as description
    FROM filtered_buildings
    GROUP BY source
    ORDER BY count DESC
""").show()


┌────────────────┬────────┬───────────────────────────────────────────────────────────┐
│     source     │ count  │                        description                        │
│    varchar     │ int64  │                          varchar                          │
├────────────────┼────────┼───────────────────────────────────────────────────────────┤
│ osm_no_overlap │ 483561 │ OSM buildings - no FKB overlap (green)                    │
│ osm_different  │   1856 │ OSM buildings - significantly different from FKB (yellow) │
│ fkb            │   1591 │ FKB buildings (red)                                       │
└────────────────┴────────┴───────────────────────────────────────────────────────────┘



In [ ]:
# Visualize filtered buildings on map
kart2 = keplergl.KeplerGl(height=700)

kart2.add_data(
    con.sql("""
        SELECT * EXCLUDE(geometry), 
               ST_AsText(geometry) as geometry 
        FROM filtered_buildings
    """).df(), 
    name='Filtered Buildings'
)

kart2
